##  The overall work flow can be seperated into 3 three section as below.

1. 透過使用者輸入的各項參數，回推這支債券的殖利率。
2. 因為一年期的債券沒有複利，所以殖利率等於即期利率，故透過此關係可以推出下一期即期利率。
3. 透過即期利率再去推遠期利率。


Note:<br>
Our goal is to generate Forward Rates between different periods of time. (As the image below shows. If you can't see the image, please refresh again.)

![](https://drive.google.com/uc?export=view&id=1XkOBw8zvAyEKaPCP17GyMF1bHbWqFXEu)

## Section 1:
透過使用者輸入的各項參數，回推這支債券的殖利率。

### Step 1:  
 Ask users to input Current Bond Price, Bond Par Value, Bond Coupon Rate, Years to Maturity, and frequency of payment per year respectively.

In [ ]:
print("What is the Current Bond Price?")
price = float(input())
print("What is the Bond Par Value?")
par = float(input())
print("What is the Bond Coupon Rate (% p.a.)")
cr = float(input())
print("How long is the Years to Maturity")
n = float(input())
print("How frequent is the Payment per year")
freq = float(input())

In [19]:
# assume the user input Current Bond Price = 1029, Bond Par Value = 1000, Bond Coupon Rate = 5%, Years to Maturity=1.5, and frequency of payment per year=2.
price=1029 #使得YTM可以是3%，跟綠角那篇文章提供的例子相同
par=1000
cr=5
n=2
freq=2

### Step 2:
Calculate YTM of the bond base on the input of the user.


In [2]:
import scipy.optimize as optimize

def bond_ytm(price, par, n, cr, freq, guess=0.05):
    freq = float(freq)
    periods = n*freq
    coupon = cr/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + par/(1+y/freq)**(freq*n) - price
        
    return optimize.newton(ytm_func, guess)

if __name__ == "__main__":
    ytm = bond_ytm(price=price, par=par, n=n, cr=cr, freq=freq)
    print(ytm)

0.034862654986864326


## Section 2:
因為一年期的債券沒有複利，所以殖利率等於即期利率，故透過此關係可以推出下一期即期利率。

### Step 3:
Calculate the spot rate.

In [5]:
import scipy.optimize as optimize

def spot(price, par, n, cr, freq, guess=0.05):
    freq = float(freq)
    periods = n*freq
    coupon = cr/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods)-1)]
    spot_func = lambda x: price - sum([coupon/(1+ytm/n/freq*t)**(freq*t) for t in dt]) - (par+coupon)/(1+x)**(freq*n)
    return optimize.newton(spot_func, guess)

if __name__ == "__main__":
    s = spot(price=price, par=par, n=n, cr=cr, freq=freq)*freq
    print(s)

0.03541966263835964


### Step 4:
Create an empty dataframe to store values.

In [15]:
# produce row index for the dataframe
row_name = list(range(int(n*freq)+1))
row_name = [x for x in row_name]
row_name = ['第'+ str(x) +'期' for x in row_name]
row_name

['第0期', '第1期', '第2期', '第3期', '第4期']

In [16]:
import pandas as pd
df2 = pd.DataFrame(index=row_name, columns=row_name)
df2 = df2.fillna(0) # with 0s rather than NaNs
df2

,第0期,第1期,第2期,第3期,第4期
第0期,0,0,0,0,0
第1期,0,0,0,0,0
第2期,0,0,0,0,0
第3期,0,0,0,0,0
第4期,0,0,0,0,0


In [17]:
for i in list(range(len(row_name))):
    for j in list(range(len(row_name))):
        if i>j:
            df2.iloc[i,j] = str('X')
        elif i==j:
            df2.iloc[i,j] = str('0')
df2

,第0期,第1期,第2期,第3期,第4期
第0期,0,0,0,0,0
第1期,X,0,0,0,0
第2期,X,X,0,0,0
第3期,X,X,X,0,0
第4期,X,X,X,X,0


### Step 5:
Store the spot rate calculated above into the first row.

In [18]:
dt = [(i+1)/freq for i in range(int(n*freq)-1)]
for i in list(range(int(n*freq))):
    if i==0:
        df2.iloc[0, int(i)] = 0
    else:
        df2.iloc[0, int(i)] = [((1+ytm/n/freq/freq)**(freq*t)-1)*freq for t in dt][int(i-1)]

df2.iloc[0,int(n*freq)] = s
df2

,第0期,第1期,第2期,第3期,第4期
第0期,0,0.00871566,0.0174693,0.0262611,0.0354197
第1期,X,0,0,0,0
第2期,X,X,0,0,0
第3期,X,X,X,0,0
第4期,X,X,X,X,0


## Section 3:
透過即期利率再去推遠期利率。

### Step 6:
Calculate the forward rate from the spot rate, and store into the dataframe.

In [11]:
for i in list(range(len(row_name))):
    for j in list(range(len(row_name))):
        if j>i and i!=0:
            df2.iloc[i,j] = freq*(((1+df2.iloc[i-1,j]/freq)**j/(1+df2.iloc[i-1,j-1]/freq)**i)**(1/(j-i))-1)

In [12]:
df2

,第0期,第1期,第2期,第3期,第4期
第0期,0,0.00871566,0.0174693,0.0262611,0.0354197
第1期,X,0,0.0262611,0.0306714,0.0384817
第2期,X,X,0,0.0395207,0.0463221
第3期,X,X,X,0,0.0668628
第4期,X,X,X,X,0
